In [1]:
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import pandas as pd
import requests
from us import states

from team import dataframe
from data.soup_helpers import get_table

In [8]:
df = dataframe.get_all_d1_schools()
dataframe.add_tournament_rankings_to_dataframe_from_csv(df, "test/data/elite8.csv")
df = dataframe.filter_none_values(df, "Tournament Ranking")
dataframe.add_ap_rankings_to_dataframe(df)
display(df)

,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking,Tournament Ranking
22,Duke University,Blue Devils,Cameron Indoor Stadium,ACC,"(44) 1955, 1960, 1963, 1964, 1966, 1978, 1979,...","(17) 1963, 1964, 1966, 1978, 1986, 1988, 1989,...","(5) 1991, 1992, 2001, 2010, 2015",13,1
31,Syracuse University,Orange,Carrier Dome,ACC,"(41) 1957, 1966, 1973, 1974, 1975, 1976, 1977,...","(6) 1975, 1987, 1996, 2003, 2013, 2016",(1) 2003,24,8
73,Villanova University,Wildcats,Finneran Pavilion,Big East,"(43) 1939, 1949, 1951, 1955, 1962, 1964, 1969,...","(7) 1939, 1971*, 1985, 2009, 2016, 2018, 2022","(3) 1985, 2016, 2018",12,2
81,University of Northern Colorado,Bears,Butler–Hancock Sports Pavilion,Big Sky,(1) 2011,(0),(0),25,2
115,University of Texas at Austin,Longhorns,Moody Center,Big 12,"(36) 1939, 1943, 1947, 1960, 1963, 1972, 1974,...","(3) 1943, 1947, 2003",(0),22,3
180,Mount St. Mary's University,Mountaineers,Knott Arena,MAAC,"(6) 1995, 1999, 2008, 2014, 2017, 2021",(0),(0),None,4
285,The Citadel,Bulldogs,McAlister Field House,Southern,(0),(0),(0),21,5
341,Gonzaga University,Bulldogs,McCarthey Athletic Center,West Coast,"(24) 1995, 1999, 2000, 2001, 2002, 2003, 2004,...","(2) 2017, 2021",(0),6,1


In [3]:
display(df.loc[df["School"] == "Gonzaga University"])
display(df.loc[df["School"] == "Syracuse University"])
display(df.loc[df["School"] == "University of Texas at Austin"])

,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking,Tournament Ranking
341,Gonzaga University,Bulldogs,McCarthey Athletic Center,West Coast,"(24) 1995, 1999, 2000, 2001, 2002, 2003, 2004,...","(2) 2017, 2021",(0),6,1


,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking,Tournament Ranking
31,Syracuse University,Orange,Carrier Dome,ACC,"(41) 1957, 1966, 1973, 1974, 1975, 1976, 1977,...","(6) 1975, 1987, 1996, 2003, 2013, 2016",(1) 2003,24,8


,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking,Tournament Ranking
115,University of Texas at Austin,Longhorns,Moody Center,Big 12,"(36) 1939, 1943, 1947, 1960, 1963, 1972, 1974,...","(3) 1943, 1947, 2003",(0),23,3


## Get Colors

In [4]:
color_table = get_table("https://en.wikipedia.org/wiki/Module:College_color", 3)
colors = list()
for i, row in enumerate(color_table.find_all("tr")[2:]):
    cdata = row.find_all("td")
    name = cdata[0].string
    color1 = cdata[1].get("style").split(":")[1]
    color2 = cdata[3].get("style").split(":")[1]
    if "transparent" in color2:
        color2 = cdata[2].get("style").split(":")[1]
    colors.append((name, color1, color2))
colors[0]

('Abilene Christian Wildcats', '#4E2683', '#C5C6C8')

## All D1 Schools Table

In [26]:
URL = "https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_institutions"
schools_table = get_table(URL, 1)
header = schools_table.find_all("tr")[1]
for i, row in enumerate(schools_table.find_all("tr")[2:]):
    data = row.find_all("td")
    school = data[0].find("a").string
    location = f"{data[3].find('a').string}, {data[4].find('a').string}"
    is_private = data[5].find("a").string.casefold() == "private".casefold()
    print(school, location, is_private)
    break

Abilene Christian University Abilene, TX True


In [10]:
data = requests.get("https://www.caninejournal.com/college-colors/")
b = BeautifulSoup(data.text)
line_break_set = {"Name", "City", "Mascot", "Nickname", "Location", "Team Name", "Home of the Catamounts", "The Island University", "Marching Highland Cavaliers", "Senators"}
for line in b.find_all('li')[46:]:
    if line and ":" in line.text:
        sp = line.text.split(":")
        school = sp[0].strip()
        colors = sp[1]
        for t in line_break_set:
            colors = colors.split(t)[0].strip()
        if "&" in colors:
            color_split = colors.split("&")
            primary_color = color_split[0].strip()
            secondary_color = color_split[1].strip()
        if "," in colors:
            color_split = colors.split(",")
            primary_color = color_split[0].strip()
            secondary_color = color_split[1].split("&")[0].strip()
        if school and school not in line_break_set:
            pass
            # print(f"{school}, {primary_color}, {secondary_color}")